# EEE6504 - HW3 #
## Jackson Cornell  | Michael Kim ##

In [1]:
import math
import time
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz

## Functions ##

In [2]:
# Gaussian kernel
def Kernel(x, x1, cov) :
    return np.exp(-np.linalg.norm(x - x1, 2, axis=1)**2 / cov**2)

def Kernel2(x, x1, cov) :
    return np.exp(-(x - x1)**2 / cov**2)

In [3]:
# KLMS filter with MSE cost function
# will edit into QKLMS later
def KLMS_MSE(x, d, kernel_size, M, mu) :
    # calculate epochs
    epochs = np.size(x) - M
    # initialize vectors
    e = np.zeros(epochs)
    y = np.zeros(epochs)
    a = np.zeros(epochs)
    growth = np.zeros(epochs)
    input_m = np.zeros((epochs, M))
    # define input matrix
    for j in range(epochs) :
        input_m[j,:] = x[j:(j+M)]
    # steepest descent
    for i in range(epochs) :
        # compute output
        y[i] = 0
        x1 = np.array(x[i:(i+M)])
        if (i != 0) :
            x2 = input_m[0:i,:]
            y[i] += a[0:i].T @ Kernel(x1, x2, kernel_size)
        # update error and coefficients
        e[i] = d[i+M-1] - y[i]
        a[i] = mu*e[i]
        growth[i] = i+1
    # return weights and learning curve
    return e**2, y, growth, a

In [ ]:
# KRLS filter with MSE cost function
# will turn into QKRLS later
def KRLS_MSE(x, d, lam, kernel_size) :
    # calculate epochs
    epochs = np.size(x)
    # initialize vectors
    e = np.zeros(epochs)
    y = np.zeros(epochs)
    a = np.zeros(epochs)
    growth = np.zeros(epochs)
    r = np.zeros(epochs)
    # iterate through samples
    for i in range(epochs) :
        # calculate vectors
        if (i == 0) :
            Q = np.array(lam + Kernel(x[0], x[0], kernel_size)).inv
            a[0] = Q*d[0]
        else :
            x_v = x[0:i]
            h = Kernel(x[i], x_v, kernel_size).T
            z = Q @ h
            r[i] = lam + Kernel(x[i], x[i], kernel_size) - z.T@h
            #Q = r.inv @ ???
            e[i] = d[i]- h[i].T @ a[i-1]
           # a[i] = ???